# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 8:55AM,242389,10,8300961.1,"Citieffe, Inc.",Released
1,Jul 21 2022 8:53AM,242388,19,ACG-2102490674,ACG North America LLC,Released
2,Jul 21 2022 8:50AM,242386,10,0085893682,ISDIN Corporation,Released
3,Jul 21 2022 8:50AM,242386,10,0085893683,ISDIN Corporation,Released
4,Jul 21 2022 8:50AM,242386,10,0085893684,ISDIN Corporation,Released
5,Jul 21 2022 8:50AM,242386,10,0085898374,ISDIN Corporation,Released
6,Jul 21 2022 8:50AM,242386,10,0085898376,ISDIN Corporation,Released
7,Jul 21 2022 8:50AM,242386,10,0085898379,ISDIN Corporation,Released
8,Jul 21 2022 8:50AM,242386,10,0085898377,ISDIN Corporation,Released
9,Jul 21 2022 8:50AM,242386,10,0085898378,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,242384,Released,12
29,242385,Released,1
30,242386,Released,57
31,242388,Released,1
32,242389,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242384,NaN,NaN,12.0
242385,NaN,NaN,1.0
242386,NaN,NaN,57.0
242388,NaN,NaN,1.0
242389,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242295,0.0,0.0,1.0
242297,0.0,0.0,1.0
242299,0.0,0.0,1.0
242305,0.0,0.0,1.0
242306,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242295,0,0,1
242297,0,0,1
242299,0,0,1
242305,0,0,1
242306,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242295,0,0,1
1,242297,0,0,1
2,242299,0,0,1
3,242305,0,0,1
4,242306,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242295,,,1
1,242297,,,1
2,242299,,,1
3,242305,,,1
4,242306,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 8:55AM,242389,10,"Citieffe, Inc."
1,Jul 21 2022 8:53AM,242388,19,ACG North America LLC
2,Jul 21 2022 8:50AM,242386,10,ISDIN Corporation
59,Jul 21 2022 8:33AM,242385,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx"
60,Jul 21 2022 8:21AM,242384,10,ISDIN Corporation
72,Jul 21 2022 8:11AM,242383,10,"Methapharm, Inc."
73,Jul 21 2022 8:11AM,242382,10,"Amcyte Pharma, Inc."
74,Jul 21 2022 8:05AM,242381,10,Emerginnova
76,Jul 21 2022 7:20AM,242380,19,"AdvaGen Pharma, Ltd"
91,Jul 20 2022 6:51PM,242378,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 21 2022 8:55AM,242389,10,"Citieffe, Inc.",,,1
1,Jul 21 2022 8:53AM,242388,19,ACG North America LLC,,,1
2,Jul 21 2022 8:50AM,242386,10,ISDIN Corporation,,,57
3,Jul 21 2022 8:33AM,242385,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",,,1
4,Jul 21 2022 8:21AM,242384,10,ISDIN Corporation,,,12
5,Jul 21 2022 8:11AM,242383,10,"Methapharm, Inc.",,,1
6,Jul 21 2022 8:11AM,242382,10,"Amcyte Pharma, Inc.",,,1
7,Jul 21 2022 8:05AM,242381,10,Emerginnova,,,2
8,Jul 21 2022 7:20AM,242380,19,"AdvaGen Pharma, Ltd",,,15
9,Jul 20 2022 6:51PM,242378,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 8:55AM,242389,10,"Citieffe, Inc.",1,,
1,Jul 21 2022 8:53AM,242388,19,ACG North America LLC,1,,
2,Jul 21 2022 8:50AM,242386,10,ISDIN Corporation,57,,
3,Jul 21 2022 8:33AM,242385,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",1,,
4,Jul 21 2022 8:21AM,242384,10,ISDIN Corporation,12,,
5,Jul 21 2022 8:11AM,242383,10,"Methapharm, Inc.",1,,
6,Jul 21 2022 8:11AM,242382,10,"Amcyte Pharma, Inc.",1,,
7,Jul 21 2022 8:05AM,242381,10,Emerginnova,2,,
8,Jul 21 2022 7:20AM,242380,19,"AdvaGen Pharma, Ltd",15,,
9,Jul 20 2022 6:51PM,242378,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 8:55AM,242389,10,"Citieffe, Inc.",1,,
1,Jul 21 2022 8:53AM,242388,19,ACG North America LLC,1,,
2,Jul 21 2022 8:50AM,242386,10,ISDIN Corporation,57,,
3,Jul 21 2022 8:33AM,242385,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",1,,
4,Jul 21 2022 8:21AM,242384,10,ISDIN Corporation,12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 8:55AM,242389,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Jul 21 2022 8:53AM,242388,19,ACG North America LLC,1.0,NaN,NaN
2,Jul 21 2022 8:50AM,242386,10,ISDIN Corporation,57.0,NaN,NaN
3,Jul 21 2022 8:33AM,242385,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",1.0,NaN,NaN
4,Jul 21 2022 8:21AM,242384,10,ISDIN Corporation,12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 8:55AM,242389,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Jul 21 2022 8:53AM,242388,19,ACG North America LLC,1.0,0.0,0.0
2,Jul 21 2022 8:50AM,242386,10,ISDIN Corporation,57.0,0.0,0.0
3,Jul 21 2022 8:33AM,242385,19,"Omnivium Pharmaceuticals, LLC DBA WDSrx",1.0,0.0,0.0
4,Jul 21 2022 8:21AM,242384,10,ISDIN Corporation,12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2665948,106.0,2.0,0.0
12,242336,1.0,0.0,0.0
16,484592,2.0,0.0,0.0
19,1938836,30.0,13.0,23.0
20,242374,5.0,0.0,0.0
21,1696490,7.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2665948,106.0,2.0,0.0
1,12,242336,1.0,0.0,0.0
2,16,484592,2.0,0.0,0.0
3,19,1938836,30.0,13.0,23.0
4,20,242374,5.0,0.0,0.0
5,21,1696490,7.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,106.0,2.0,0.0
1,12,1.0,0.0,0.0
2,16,2.0,0.0,0.0
3,19,30.0,13.0,23.0
4,20,5.0,0.0,0.0
5,21,7.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,106.0
1,12,Released,1.0
2,16,Released,2.0
3,19,Released,30.0
4,20,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,23.0,0.0,0.0
Executing,2.0,0.0,0.0,13.0,0.0,0.0
Released,106.0,1.0,2.0,30.0,5.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,2.0,0.0,0.0,13.0,0.0,0.0
2,Released,106.0,1.0,2.0,30.0,5.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,23.0,0.0,0.0
1,Executing,2.0,0.0,0.0,13.0,0.0,0.0
2,Released,106.0,1.0,2.0,30.0,5.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()